reference:
- https://www.kaggle.com/code/toshikazuwatanabe/connect4-make-submission-with-stable-baselines3
- https://www.kaggle.com/code/kubamaliszewski/connectx-ppo

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from torch import nn

from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import load_results
from stable_baselines3.common.torch_layers import NatureCNN
from stable_baselines3.common.policies import ActorCriticPolicy, ActorCriticCnnPolicy

In [ ]:
LOG_DIR = os.path.join(os.getcwd(), 'log')	# トレーニングのログを保存するディレクトリ
os.makedirs(LOG_DIR, exist_ok=True)

環境

In [ ]:
# 環境の作成
from environment import ConnectFourGym

training_env = ConnectFourGym(opponent='random')
training_env

In [ ]:
# ログを取得する
training_env = Monitor(training_env, LOG_DIR, allow_early_resets=True)
training_env

In [ ]:
# 「DummyVecEnv」は、OpenAI Gymの環境をベクトル化するための特殊なラッパーです。
# 通常、強化学習アルゴリズムは一度に1つの環境しか処理できませんが、これを使用することで
# 複数の環境を同時に実行することができます。これにより、学習プロセスが効率的になります。
training_env = DummyVecEnv([lambda: training_env])
training_env

In [ ]:
training_env.observation_space.sample()

モデル

In [ ]:
from agent import Net